In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
import pandas as pd
import numpy as np
import re
from scipy.sparse import hstack
import warnings
warnings.filterwarnings("ignore")

##  1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv

In [2]:
df = pd.read_csv('salary-train.csv')
df.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


##  2. Проведите предобработку:

In [3]:
# Приведите тексты к нижнему регистру.
df['FullDescription'] = [x.lower() for x in df['FullDescription']]

# Замените все, кроме букв и цифр, на пробелы это облегчит дальнейшее разделение 
# текста на слова. Для такой замены в строке text подходит следующий вызов:
df['FullDescription'] = [re.sub('[^a-z0-9]', ' ', x) for x in df['FullDescription']]

In [4]:
# Примените TfidfVectorizer для преобразования текстов в векторы признаков. 
# Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).
model = TfidfVectorizer(min_df=5)
X = model.fit_transform(df['FullDescription'])

In [5]:
# Замените пропуски в столбцах LocationNormalized и ContractTime
# на специальную строку ’nan’. Код для этого был приведен выше.

df['LocationNormalized'].fillna('nan', inplace=True)
df['ContractTime'].fillna('nan', inplace=True)

In [6]:
# Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.
enc = DictVectorizer()

X_1hot = enc.fit_transform(
    df[['LocationNormalized', 'ContractTime']].to_dict('records')
)

In [7]:
# Объедините все полученные признаки в одну матрицу "объекты признаки". 
# Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. 
# Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.
X_ = hstack((X, X_1hot))

 ## 3. Обучите гребневую регрессию с параметром alpha=1. Целевая переменная записана в столбце SalaryNormalized

In [8]:
y = df["SalaryNormalized"]
mdl = Ridge(alpha=1, random_state=123)
mdl.fit(X_, y)


Ridge(alpha=1, random_state=123)

 ## 4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv.
 Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [9]:
df_test = pd.read_csv('10-statement-linreg/salary-test-mini.csv')

X_test = model.transform([re.sub('[^a-z0-9]', ' ', x.lower()) for x in df_test['FullDescription']])
X_1htest = enc.transform(df_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

X_t = hstack((X_test, X_1htest))
predict = mdl.predict(X_t)
print(*[round(x, 2) for x in predict])

56577.47 37200.47
